In [58]:
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

In [59]:
DATASET_PATH = "dataset.json"

In [60]:
#Loading the dataset

with open(DATASET_PATH, "r") as fp:
    data = json.load(fp)
X = np.array(data['MFCCs'])
y = np.array(data['labels'])
print("Number of datasets: ", len(X) if len(X)==len(y) else "Number of MFCCs and labels don't match")

Number of datasets:  120


In [61]:
#Training, test, and validation data splitting

Xtr, Xt, ytr, yt = train_test_split(X, y, test_size=0.1)
Xtr, Xval, ytr, yval = train_test_split(Xtr, ytr, test_size=0.1)

Xtr = Xtr[..., np.newaxis]
Xt = Xt[..., np.newaxis]
Xval = Xval[..., np.newaxis]

input_shape = (Xtr.shape[1], Xtr.shape[2], 1)
print(input_shape)

(126, 13, 1)


In [62]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.8):
          self.model.stop_training = True

callbacks = myCallback()

In [63]:
model = tf.keras.models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l1(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'),
    layers.Dropout(0.2),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'),
    layers.Dropout(0.2),
    layers.Conv2D(32, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 124, 11, 64)       640       
_________________________________________________________________
batch_normalization_26 (Batc (None, 124, 11, 64)       256       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 62, 6, 64)         0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 62, 6, 64)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 60, 4, 32)         18464     
_________________________________________________________________
batch_normalization_27 (Batc (None, 60, 4, 32)         128       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 30, 2, 32)       

In [64]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=10e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [65]:
history = model.fit(
    Xtr,
    ytr,
    epochs=10000,
    validation_data=(Xval, yval),
    callbacks=[callbacks]
)

/10000
4/4 [==============================] - 0s 33ms/step - loss: 0.2996 - accuracy: 0.9794 - val_loss: 2.5316 - val_accuracy: 0.4545
Epoch 9352/10000
4/4 [==============================] - 0s 36ms/step - loss: 0.3198 - accuracy: 0.9691 - val_loss: 2.6111 - val_accuracy: 0.4545
Epoch 9353/10000
4/4 [==============================] - 0s 28ms/step - loss: 0.3463 - accuracy: 0.9485 - val_loss: 2.6261 - val_accuracy: 0.4545
Epoch 9354/10000
4/4 [==============================] - 0s 30ms/step - loss: 0.2863 - accuracy: 0.9897 - val_loss: 2.7122 - val_accuracy: 0.4545
Epoch 9355/10000
4/4 [==============================] - 0s 34ms/step - loss: 0.3293 - accuracy: 0.9588 - val_loss: 2.6785 - val_accuracy: 0.4545
Epoch 9356/10000
4/4 [==============================] - 0s 37ms/step - loss: 0.3045 - accuracy: 0.9794 - val_loss: 2.7415 - val_accuracy: 0.4545
Epoch 9357/10000
4/4 [==============================] - 0s 32ms/step - loss: 0.2689 - accuracy: 0.9897 - val_loss: 2.9133 - val_accuracy: 0.

In [67]:
model.save("model.h5")